In [1]:
#!pip install matplotlib

from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
    ServiceContext
)
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display
from llama_index.embeddings.ollama import OllamaEmbedding
import faiss
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.response.notebook_utils import display_source_node

/Users/rspamzn/Documents/DevAx/Trainings/NN/agentic-ai/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [13]:
# load documents
documents = SimpleDirectoryReader(input_files=["data/thinking_machines.txt"]).load_data()
embed_model = OllamaEmbedding(
    model_name="mxbai-embed-large:latest"
)
service_context = ServiceContext.from_defaults(llm=None, embed_model=embed_model)
splitter = SemanticSplitterNodeParser(buffer_size=1, breakpoint_percentail_thresholed=95,
                                      embed_model=embed_model)
nodes = splitter.get_nodes_from_documents(documents)

/var/folders/zp/lgmyv6v166l35bqv9zb52bqc0000gr/T/ipykernel_30247/3070529811.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=None, embed_model=embed_model)


LLM is explicitly disabled. Using MockLLM.


In [14]:
for i, node in enumerate(nodes):
    print(i, " ---- ", node) #node.get_content()

0  ----  Node ID: a37a542d-341c-4467-8c66-f2a8f1e1b3b1
Text: When I was an entrepreneur about a decade ago, I landed on a
golden use case (or so I fantasised). It is to apply Natural Language
Processing (NLP) to convert english statements into structured facts
(Subject-Predicate-Object) that adhere to commonly-agreed Domain
Ontologies. For example, “Singapore’s economic inflation is estimated
at 4.5%” can...
1  ----  Node ID: a62ef461-d24f-4306-a2b9-d0a3fa446a65
Text: And the 3rd and the 4th are fundamentally similar in the
approach. Therefore we can now reduce the categories into two to move
forward — 1/symbol manipulation and; 2/sequence transduction using
pattern matching. Now, it is important to understand the limitations
of both these approaches to avoid rude shocks. In the first symbol
manipulation appr...
2  ----  Node ID: c728ca39-9028-4c9d-972f-ff80080fc9e5
Text: The semantic link missing in this statement is that one needs to
buy tickets to get access to the movies. This is o

In [15]:
faiss_index = faiss.IndexFlatL2(1024)
faiss_index.is_trained

True

In [16]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(
    nodes, storage_context=storage_context,
    service_context=service_context
)

In [17]:
query_engine = index.as_query_engine()
response = query_engine.query(
    "What is the author's current job scope?"
)

In [18]:
for n in response.source_nodes:
    display_source_node(n, source_length=20000)

**Node ID:** 37871ead-d804-479d-9cac-44e08b9b54f7<br>**Similarity:** 290.5322265625<br>**Text:** Before that, I want to dive into making this article a little more contextual for developers. My current job scope is to augment developer productivity and in that scope, code generation using GenAI is an important weapon in any developers’ armoury. And we developers need to know when and where to apply this technology safely!

Back to the 2 challenges of hallucination and lacking causality, the first problem is easy in the development domain. The generated code from the models can be easily fact checked — one just have to execute them. That is not hard. Most boiler plate codes have been working well in my tests. The hallucination flaws start to appear when you prompt for the not-so-common patterns (ex. generate a neural net algorithm for x inputs, y hidden nodes, z outputs using a certain activation function). In general application development, such requirements are rare and therefore, using the models to generate the codes do result in 50%+ productivity. The causality challenge is the worst one. In programming, often the decisions made at one part of the code will have multiple hops of connections to the rest. Imagine a state stored in a data member in an object that needs to be reset before a common algorithm in a method is executed. If the developer misses that (the cause) the program spits out bad results (the effect). If such a code is given to the generative models and asked for recommendations to fix it, they predictably struggle all the time. That is the hypothesis and we need to validate this to declare this as the challenging anti-pattern for applying generative models in programming.

To prove the hypothesis, I took the case of coding a neural network itself. There is a poetic beauty in this scenario — to get a neural network (GenAI) to produce or fix another neural network. Self Replicating Machines, Wow ! To keep the problem simple, I coded the 2 hidden node network from a Josh Starmer video (btw, if you want to refresh the ML foundations, Josh has an amazingly intuitive guide).

I purposefully avoided using the matrices and dot products so that I can create the problem scenario. This is the faulty code that will not converge after back propagation, despite any epoch size.

I have coded a piece of langchain client that will invoke OpenAI APIs to get a recommended fix for this code.<br>

**Node ID:** 2ec38729-9d15-4e46-a2d1-da5c860414d2<br>**Similarity:** 294.3992004394531<br>**Text:** Disclaimer : I could be wrong here and I had been wrong before :)<br>

In [19]:
index.storage_context.persist(persist_dir="index")

In [20]:
vector_store = FaissVectorStore.from_persist_path("./index/default__vector_store.json")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./index"
)
retrieved_index = load_index_from_storage(storage_context=storage_context, service_context=service_context)

In [21]:
retriever = retrieved_index.as_retriever()
response = retriever.retrieve(
    "What is the author's current job scope?"
)

In [22]:
for n in response:
    display_source_node(n, source_length=20000)

**Node ID:** 37871ead-d804-479d-9cac-44e08b9b54f7<br>**Similarity:** 290.5322265625<br>**Text:** Before that, I want to dive into making this article a little more contextual for developers. My current job scope is to augment developer productivity and in that scope, code generation using GenAI is an important weapon in any developers’ armoury. And we developers need to know when and where to apply this technology safely!

Back to the 2 challenges of hallucination and lacking causality, the first problem is easy in the development domain. The generated code from the models can be easily fact checked — one just have to execute them. That is not hard. Most boiler plate codes have been working well in my tests. The hallucination flaws start to appear when you prompt for the not-so-common patterns (ex. generate a neural net algorithm for x inputs, y hidden nodes, z outputs using a certain activation function). In general application development, such requirements are rare and therefore, using the models to generate the codes do result in 50%+ productivity. The causality challenge is the worst one. In programming, often the decisions made at one part of the code will have multiple hops of connections to the rest. Imagine a state stored in a data member in an object that needs to be reset before a common algorithm in a method is executed. If the developer misses that (the cause) the program spits out bad results (the effect). If such a code is given to the generative models and asked for recommendations to fix it, they predictably struggle all the time. That is the hypothesis and we need to validate this to declare this as the challenging anti-pattern for applying generative models in programming.

To prove the hypothesis, I took the case of coding a neural network itself. There is a poetic beauty in this scenario — to get a neural network (GenAI) to produce or fix another neural network. Self Replicating Machines, Wow ! To keep the problem simple, I coded the 2 hidden node network from a Josh Starmer video (btw, if you want to refresh the ML foundations, Josh has an amazingly intuitive guide).

I purposefully avoided using the matrices and dot products so that I can create the problem scenario. This is the faulty code that will not converge after back propagation, despite any epoch size.

I have coded a piece of langchain client that will invoke OpenAI APIs to get a recommended fix for this code.<br>

**Node ID:** 2ec38729-9d15-4e46-a2d1-da5c860414d2<br>**Similarity:** 294.3992004394531<br>**Text:** Disclaimer : I could be wrong here and I had been wrong before :)<br>